# AI-Powered Personal Chatbot with Quality Control

## Project Overview
This notebook demonstrates how to build an intelligent chatbot that:
- Acts as your digital representative using your LinkedIn profile and personal summary
- Answers questions about your career, skills, and experience
- Includes an AI quality control system that evaluates and improves responses
- Uses Gradio for an interactive web interface

## Tech Stack
- **OpenAI GPT-4o-mini**: Main chatbot model
- **Groq (GPT-OSS-120B)**: Quality evaluator model
- **Gradio**: Interactive web UI
- **PyPDF**: PDF parsing for LinkedIn profile
- **Pydantic**: Structured output validation

---
## Step 1: Import Required Libraries

We'll need:
- `dotenv`: To load environment variables (API keys)
- `OpenAI`: To interact with OpenAI and Groq APIs
- `PdfReader`: To extract text from PDF files
- `gradio`: To create an interactive chat interface

In [3]:
# Import necessary packages
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

---
## Step 2: Initialize OpenAI Client

Load environment variables from `.env` file and create an OpenAI client instance.
Make sure you have `OPENAI_API_KEY` in your `.env` file.

In [4]:
# Load environment variables from .env file
# override=True ensures the latest values are loaded
load_dotenv(override=True)

# Initialize OpenAI client (API key is loaded from environment)
openai = OpenAI()

---
## Step 3: Extract LinkedIn Profile from PDF

We'll read a LinkedIn profile PDF and extract all the text content.
This will be used to give the chatbot context about your professional background.

In [5]:
# Read the LinkedIn profile PDF file
reader = PdfReader("me/linkedin.pdf")

# Initialize an empty string to store all extracted text
linkedin = ""

# Loop through each page in the PDF
for page in reader.pages:
    # Extract text from the current page
    text = page.extract_text()
    
    # If text exists, append it to our linkedin string
    if text:
        linkedin += text

### Preview the Extracted LinkedIn Content

In [6]:
# Display the extracted LinkedIn profile text
print(linkedin)

   
Contact
vishnudharshank@gmail.com
www.linkedin.com/in/vishnu-
dharshan-k (LinkedIn)
Top Skills
Agentic AI Development
Amazon Web Services (AWS)
PySpark 
Vishnu dharshan K
Program Analyst @ CTS | Exploring AI Agents & Automations -
RAG , MCP , Langchain , Vector DB | Python • MySQL • AWS Cloud
• PySpark
Theni, Tamil Nadu, India
Summary
Working at Cognizant (CTS) with hands-on experience in MySQL,
Python, Linux, and AWS Cloud, PySpark, Snowflake. I am always
eager to learn new technologies, solve problems, and grow further in
my career while delivering real value to the organization.
Experience
Cognizant
1 year 4 months
Program Analyst
November 2025 - Present (4 months)
Kochi, Kerala, India
Program Analyst Trainee
November 2024 - October 2025 (1 year)
Kochi, Kerala, India
Education
VSB Engineering College - India
Bachelor of Engineering - BE, Electrical, Electronics and Communications
Engineering · (April 2020 - May 2024)
  Page 1 of 1


---
## Step 4: Load Personal Summary

Read a custom summary file that provides additional context about your career goals and interests.

In [7]:
# Read the personal summary from a text file
# This provides additional context beyond the LinkedIn profile
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

---
## Step 5: Define Your Name

Set the name that the chatbot will represent.

In [8]:
# Define the name of the person the chatbot represents
name = "VishnuDharshan K"

---
## Step 6: Create the System Prompt

This is the most important part! The system prompt tells the AI:
- Who it's representing
- What information it has access to
- How it should behave
- What tone to use

In [9]:
# Build the system prompt that defines the chatbot's behavior and context
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

# Add the summary and LinkedIn profile to the system prompt
system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"

# Final instruction to stay in character
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


### Preview the Complete System Prompt

In [10]:
# Display the complete system prompt to verify it looks correct
system_prompt

"You are acting as VishnuDharshan K. You are answering questions on VishnuDharshan K's website, particularly questions related to VishnuDharshan K's career, background, skills and experience. Your responsibility is to represent VishnuDharshan K for interactions on the website as faithfully as possible. You are given a summary of VishnuDharshan K's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nVishnu Dharshan K is a Program Analyst at Cognizant (CTS) in Kochi, Kerala, with a Bachelor's in Electrical, Electronics, and Communications Engineering from VSB Engineering College (2020-2024). Currently focused on the Data Engineering side, I'm honing my skills in PySpark while exploring AI agents and automations, including RAG, MCP, LangChain, and Vector DBs—I'm particularly eager to build small agents an

---
## Step 7: Create Basic Chat Function (Version 1)

This is a simple chatbot without quality control.
It takes a message and chat history, sends it to OpenAI, and returns the response.

In [11]:
# Simple chat function - no quality control yet
def chat(message, history):
    """
    Basic chat function that:
    1. Combines system prompt, chat history, and new message
    2. Sends to OpenAI API
    3. Returns the AI's response
    
    Args:
        message (str): The user's latest message
        history (list): Previous conversation history
    
    Returns:
        str: The chatbot's response
    """
    # Build the messages array: system prompt + history + new message
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    # Send to OpenAI and get response
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    
    # Extract and return the text content
    return response.choices[0].message.content

### 📝 Important Note for Non-OpenAI Users

Some providers like Groq might give an error on the second message.

This is because Gradio adds extra fields to the history object. OpenAI doesn't mind, but other models complain.

**Solution:** Add this line at the start of your `chat()` function:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

This cleans up the history variable before sending it to the API.

---
## Step 8: Launch Basic Chatbot Interface

This creates a web interface where you can interact with your AI chatbot.
It will run on http://127.0.0.1:7860

In [12]:
# Launch the Gradio chat interface
# type="messages" ensures proper formatting for OpenAI API
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


---
---
# 🎯 PART 2: Adding Quality Control System

Now we'll add an AI evaluator that checks if responses are good quality.
If a response fails evaluation, the system will automatically regenerate it!

This is an advanced agentic AI pattern.

---
## Step 9: Import Pydantic for Structured Outputs

Pydantic helps us get structured, validated responses from the AI evaluator.

In [13]:
# Import BaseModel from Pydantic for structured output validation
from pydantic import BaseModel

---
## Step 10: Define Evaluation Response Structure

We want the evaluator to return:
- A boolean: Is the response acceptable?
- Feedback: Why or why not?

In [14]:
# Define the structure for evaluation responses using Pydantic
class Evaluation(BaseModel):
    """
    Structure for the AI evaluator's response.
    
    Attributes:
        is_acceptable (bool): Whether the chatbot's response passes quality check
        feedback (str): Explanation of why it passed or failed
    """
    is_acceptable: bool  # True if response is good, False if it needs improvement
    feedback: str        # Detailed feedback about the response quality

---
## Step 11: Create Evaluator System Prompt

This defines how the quality control AI should judge responses.

In [15]:
# System prompt for the AI evaluator
# This AI's job is to judge whether responses are professional and accurate
evaluator_system_prompt = f"""
You are a quality control agent evaluating responses from an AI chatbot representing {name}.

Your job is to determine if the Agent's response is:
1. **Accurate** - Based on the provided background information
2. **Professional** - Appropriate for a career website
3. **Engaging** - Friendly but not overly casual
4. **On-topic** - Actually answers the user's question

Respond with:
- is_acceptable: true/false
- feedback: Explanation of your decision

Be strict but fair. If the response is unprofessional, irrelevant, or contains 
gibberish/nonsense, mark it as unacceptable.
"""

---
## Step 12: Create Evaluator User Prompt Function

This function formats the conversation context for the evaluator to review.

In [16]:
# Function to build the user prompt for the evaluator
def evaluator_user_prompt(reply, message, history):
    """
    Constructs a prompt for the evaluator AI with full conversation context.
    
    Args:
        reply (str): The chatbot's response to be evaluated
        message (str): The user's latest message
        history (list): Previous conversation history
    
    Returns:
        str: Formatted prompt for the evaluator
    """
    # Build the evaluation prompt with all necessary context
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

---
## Step 13: Initialize Groq Client for Evaluation

We'll use Groq's fast open-source model for evaluation.
Make sure you have `GROQ_API_KEY` in your `.env` file.

In [26]:
import os 

# Option 1: Use Google's Gemini for evaluation (commented out)
# gemini = OpenAI(
#     api_key = os.getenv("GOOGLE_API_KEY"),
#     base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
# )

# Option 2: Use Groq for fast, free evaluation (current choice)
groq = OpenAI(
    api_key = os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

---
## Step 14: Create the Evaluation Function

This function sends a response to the evaluator AI and gets structured feedback.

In [27]:
# Function to evaluate a chatbot response
def evaluate(reply, message, history) -> Evaluation:
    """
    Sends a chatbot response to the evaluator AI for quality check.
    
    Args:
        reply (str): The chatbot's response to evaluate
        message (str): The user's message
        history (list): Conversation history
    
    Returns:
        Evaluation: Structured response with is_acceptable and feedback
    """
    # Build messages for the evaluator
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]
    
    # Call Groq with structured output (returns Evaluation object)
    response = groq.beta.chat.completions.parse(
        model="openai/gpt-oss-120b",
        messages=messages,
        response_format=Evaluation  # Ensures response matches Evaluation structure
    )
    
    # Return the parsed Evaluation object
    return response.choices[0].message.parsed

---
## Step 15: Test the Evaluator

Let's test the evaluation system with a sample question and response.

In [ ]:
# Test: Generate a response to a sample question
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "do you hold a patent?"}
]

# Get response from chatbot
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [19]:
# Display the chatbot's response
reply

"I do not currently hold a patent. My focus at this stage in my career has been on building skills in data engineering and exploring AI technologies. However, I'm always open to innovative projects and ideas, and I look forward to the possibility of contributing to advancements in the future. If you have any questions about my work or background, feel free to ask!"

In [28]:
# Evaluate the response
# This will return an Evaluation object with is_acceptable and feedback
evaluate(reply, "do u hold patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response accurately answers the user's question based on the provided background (no patents mentioned), maintains a professional and engaging tone, and invites further questions. It aligns well with the persona of VishnuDharshan K.")

---
## Step 16: Create Rerun Function

If a response fails evaluation, this function regenerates it with feedback.
This is the "self-correction" mechanism!

In [29]:
# Function to regenerate a response when it fails quality check
def rerun(reply, message, history, feedback):
    """
    Regenerates a chatbot response after it failed evaluation.
    
    The system prompt is updated to include:
    - The rejected response
    - The reason it was rejected
    
    This helps the AI learn what NOT to do.
    
    Args:
        reply (str): The failed response
        message (str): The user's message
        history (list): Conversation history
        feedback (str): Why the response was rejected
    
    Returns:
        str: New improved response
    """
    # Create an updated system prompt that includes the rejection feedback
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    
    # Build new messages with the updated system prompt
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    
    # Generate a new response (using Groq for speed)
    response = groq.chat.completions.create(model="openai/gpt-oss-120b", messages=messages)
    
    # Return the improved response
    return response.choices[0].message.content

---
## Step 17: Enhanced Chat Function with Quality Control

This is the final version that includes:
1. Generate response
2. Evaluate response
3. If it fails, regenerate with feedback
4. Return the final approved response

**Bonus feature:** If the user mentions "patent", the system forces the chatbot to respond in Pig Latin (to test the evaluator)!

In [32]:
# Enhanced chat function with built-in quality control
def chat(message, history):
    """
    Advanced chat function with quality control loop:
    1. Generate initial response
    2. Evaluate the response
    3. If rejected, regenerate with feedback
    4. Return the approved response
    
    Args:
        message (str): User's message
        history (list): Conversation history
    
    Returns:
        str: Quality-approved response
    """
    
    # TESTING FEATURE: Force pig latin response if "patent" is mentioned
    # This tests if the evaluator catches gibberish responses
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt

    # Build messages array with appropriate system prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    
    # Generate initial response from OpenAI
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    # Send the reply to the evaluator for quality check
    evaluation = evaluate(reply, message, history)

    # Check evaluation result
    if evaluation.is_acceptable:
        # Response passed - return it
        print("✅ Passed evaluation - returning reply")
    else:
        # Response failed - regenerate with feedback
        print("❌ Failed evaluation - retrying")
        print(f"Feedback: {evaluation.feedback}")
        
        # Generate improved response using the feedback
        reply = rerun(reply, message, history, evaluation.feedback)
    
    return reply

---
## Step 18: Launch Enhanced Chatbot with Quality Control 🚀

This is the final version with automatic quality checking!

**Try asking about "patent" to see the quality control in action:**
- First, it will generate a pig latin response (gibberish)
- The evaluator will reject it
- It will automatically regenerate a proper response

All of this happens automatically behind the scenes!

In [ ]:
# Launch the final chatbot with quality control
# Watch the console output to see evaluation results in real-time!
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


❌ Failed evaluation - retrying
Feedback: The reply is unintelligible pig‑Latin gibberish, does not address the user's question about patents, and is unprofessional. It fails to represent Vishnu Dharshan K appropriately. The response should be clear, professional, and either state whether a patent is held or politely explain that none are held.
✅ Passed evaluation - returning reply


---
---
# 🎉 Congratulations!

You've built an AI chatbot with:
✅ Personalized responses based on your LinkedIn profile  
✅ Professional tone suitable for career websites  
✅ Automatic quality control and self-correction  
✅ Interactive web interface  

## Next Steps:
1. Replace the PDF with your own LinkedIn profile
2. Update the summary.txt with your information
3. Customize the system prompt to match your personality
4. Deploy this to a real website!

## Key Concepts Learned:
- System prompts and persona engineering
- Multi-agent AI systems (chatbot + evaluator)
- Structured outputs with Pydantic
- Self-correcting AI loops
- Gradio web interfaces

---
**Created by:** VishnuDharshan K  
**GitHub:** [Your GitHub Link]  
**LinkedIn:** [linkedin.com/in/vishnu-dharshan-k](https://linkedin.com/in/vishnu-dharshan-k)

If you found this helpful, give it a ⭐ on GitHub!